In [ ]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

def generate_main_seq():
    rand_nums=[randint(0, 9) for _ in range(25*700)]
    return rand_nums
        
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 9) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique=10):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# generate data for the lstm
def generate_data():
	# generate sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# create lag inputs
	df = DataFrame(encoded)
	df = concat([df.shift(4), df.shift(3), df.shift(2), df.shift(1), df], axis=1)
	# remove non-viable rows
	values = df.values
	values = values[5:,:]
	# convert to 3d for input
	X = values.reshape(len(values), 5, 10)
	# drop last value from y
	y = encoded[4:-1,:]
	return X, y

def work():
    # define model
    model = Sequential()
    model.add(LSTM(50, batch_input_shape=(5, 5, 10), stateful=True))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    # fit model
    for i in range(500):
        print(i)
        X, y = generate_data()
        model.fit(X, y, epochs=1, batch_size=5, verbose=2, shuffle=False)
        model.reset_states()

    # evaluate model on new data
    X, y = generate_data()
    yhat = model.predict(X, batch_size=5)
    print('Expected:  %s' % one_hot_decode(y))
    print('Predicted: %s' % one_hot_decode(yhat))

#original fit-predict
#work()

In [ ]:
seq=25
f = generate_main_seq()
Xg=[]
yg=[]
for i in range(10, 700):
    #print(f[rec:rec+seq])
    X_cur = f[i:i+seq]
    Xg.append(X_cur)
    
    k=i+1
    
    y_cur = f[k:k+seq]
    yg.append(y_cur)
    
    #y=f[i+seq]
    #print('x:',X_cur)
    #print('y:',y_cur)
    #print('=')

In [ ]:
# one hot encode sequence
def one_hot_encode(sequence, n_unique=10):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

def generate_data2(Xg, yg, i):
    sequenceX = Xg[i]
    #print('x:',sequenceX)
    encodedX = one_hot_encode(sequenceX)
    dfx = DataFrame(encodedX)
    dfx = concat([dfx.shift(4), dfx.shift(3), dfx.shift(2), dfx.shift(1), dfx], axis=1)
    valuesx = dfx.values
    valuesx = valuesx[5:,:]
    X = valuesx.reshape(len(valuesx), 5, 10)

    sequencey=yg[i]
    #print('y:',sequencey)
    encodedy = one_hot_encode(sequencey)
    y = encodedy[5:,:]

    return X, y, sequenceX, sequencey

def encode(sequenceX):
    encodedX = one_hot_encode(sequenceX)
    dfx = DataFrame(encodedX)
    dfx = concat([dfx.shift(4), dfx.shift(3), dfx.shift(2), dfx.shift(1), dfx], axis=1)
    valuesx = dfx.values
    valuesx = valuesx[5:,:]
    X = valuesx.reshape(len(valuesx), 5, 10)
    return X

In [ ]:
model = Sequential()
model.add(LSTM(100, batch_input_shape=(5, 5, 10), stateful=True))
#model.add(Dense(50, activation='softmax'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
for k in range(0,1000):
  i=0
  while i<600:
    X, y, sequenceX, sequencey = generate_data2(Xg, yg, i)
    #print(X)
    #print(y)
    #print(len(X))
    #print(len(y))
    #print('x:',one_hot_decode(X))
    #print('y:',one_hot_decode(y))
    print(k, i)
    r = model.fit(X, y, epochs=1, batch_size=5, verbose=2, shuffle=False)
    model.reset_states()
    i+=1

In [ ]:
model.save('model_100x500rand25_2hidden')

In [ ]:
def new_from_last(predicted, inp_x_full):
    new_value=predicted[-1:]
    print(new_value)
    answer = inp_x_full.copy()
    answer.append(new_value[0])
    return answer[1:]

import pandas as pd
z=400 #берем 400й элемент из основного набора рандомов

X, y, origX, origY = generate_data2(Xg, yg, z)
print('=')
print('dec x:', one_hot_decode(X))
print('dec y:', one_hot_decode(y))
print('=')
yhat = model.predict(X, batch_size=5)
pred = one_hot_decode(yhat)
print(' pred:', pred)
print(' expc:', one_hot_decode(y))
print('-'*20)
n=0
prev_x=origX

def main_func(inp_pred, inp_prev_x):
    new_x=new_from_last(inp_pred, inp_prev_x)
    print('prevx:',inp_prev_x)
    print('new x:',new_x)
    encodedX=encode(new_x)
    yhat = model.predict(encodedX, batch_size=5)
    pred = one_hot_decode(yhat)
    print(' pred:', pred)
    print('%')
    return pred, new_x

n=0
x = origX
while True:
    pred, x  = main_func(pred, x)
    pred, x  = main_func(pred, x)
    pred, x  = main_func(pred, x)
    n+=1
    if n==200: break

In [ ]:
%matplotlib inline
import pandas as pd
X, y, a, b = generate_data2(Xg, yg, 400)
print('x:', one_hot_decode(X))
print('y:', one_hot_decode(y))
print('=')
yhat = model.predict(X, batch_size=5)
#print(yhat)
print('pred:', one_hot_decode(yhat))
print('expc:', one_hot_decode(y))
#print(pd.DataFrame(one_hot_decode(yhat))-pd.DataFrame(one_hot_decode(y)))
#pd.DataFrame(one_hot_decode(yhat)).plot()
#pd.DataFrame(one_hot_decode(y)).plot()

x: [9, 5, 2, 8, 9, 4, 3, 7, 3, 0, 9, 7, 4, 5, 6, 7, 0, 0, 4, 8]
y: [4, 3, 7, 3, 0, 9, 7, 4, 5, 6, 7, 0, 0, 4, 8, 4, 6, 6, 3, 4]
=



In [ ]:
#score, acc = model.evaluate(x_test, y_test,
#                            batch_size=batch_size)
#print('Test score:', score)
#print('Test accuracy:', acc)

In [ ]:
prevx=[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

pred=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
pred, x = main_func(pred, prevx)
print(pred)
print(x)
#print(pred)
#encodedX=encode(new_x)
#yhat = model.predict(encodedX, batch_size=5)
#pred = one_hot_decode(yhat)
#print(' pred:',pred)